In [54]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json
import ast
import requests
from bs4 import BeautifulSoup

In [61]:
df = pd.read_csv('../2_clean_data/combined_data.csv')

In [62]:
#Columns with lists are represented as strings, so this is converting them into list/np.array types

for index, row in df.iterrows():
    df.iloc[index]['salaries'] = ast.literal_eval(df.iloc[index]['salaries'])
    df.iloc[index]['semester'] = ast.literal_eval(df.iloc[index]['semester'])
    df.iloc[index]['years_taught'] = ast.literal_eval(df.iloc[index]['years_taught'])
    df.iloc[index]['course'] = ast.literal_eval(df.iloc[index]['course'])
    df.iloc[index]['average_rating'] = np.fromstring(df.iloc[index]['average_rating'].strip("[]"), sep=',')
    df.iloc[index]['num_reviews'] = np.fromstring(df.iloc[index]['num_reviews'].strip("[]"), sep=',')
    df.iloc[index]['average_gpa'] = np.fromstring(df.iloc[index]['average_gpa'].strip("[]"), sep=',')
    df.iloc[index]['num_students'] = ast.literal_eval(df.iloc[index]['num_students'])

In [63]:
df = df.explode(['course', 'semester', 'average_rating', 'num_reviews', 'average_gpa', 'num_students'])

In [64]:
#Unique departments (first 4 letters of course)

uniqueCourses = df['course'].str[0:4].unique()

print(df['course'].str[0:4].unique())
print(len(df['course'].str[0:4].unique()))

['PSYC' 'ASTR' 'CMSC' 'ECON' 'HESI' 'EDHI' 'COMM' 'INST' 'ENAE' 'ENME'
 'ENMA' 'ENRE' 'ARTH' 'PHYS' 'STAT' 'ENGL' 'PHSC' 'ANTH' 'BIOE' 'GEMS'
 'MATH' 'BMGT' 'BULM' 'CMLT' 'GERM' 'JWST' 'ISRL' 'BUSO' 'DANC' 'ARHU'
 'TDPS' 'UNIV' 'ENEE' 'PERS' 'ARAB' 'ENCE' 'HIST' 'HACS' 'CCJS' 'GEOL'
 'FIRE' 'SOCY' 'CPSD' 'CPSP' 'PLCY' 'AASP' 'HNUH' 'WMST' 'BUFN' 'BUSI'
 'ENTS' 'MSML' 'HONR' 'PUAF' 'JOUR' 'PHIL' 'LING' 'BSCI' 'BIOL' 'AOSC'
 'HDCC' 'LGBT' 'EDHD' 'EDUC' 'TLTC' 'EDMS' 'CHEM' 'CPMS' 'ENES' 'HESP'
 'HEBR' 'GEOG' 'LARC' 'EDCP' 'IDEA' 'FMSC' 'NEUR' 'MIEH' 'BUMK' 'ENST'
 'ANSC' 'INAG' 'CHBE' 'ENCH' 'SPAN' 'PORT' 'USLT' 'SLLC' 'ENTM' 'BUSM'
 'FREN' 'AMST' 'KNES' 'MLSC' 'THET' 'CHIN' 'EALL' 'BUMO' 'EDCI' 'TLPL'
 'ENPM' 'AREC' 'BISI' 'CBMG' 'ENSP' 'GVPT' 'PHPE' 'ENNU' 'HLSA' 'HLTH'
 'BSOS' 'SPHL' 'BUDT' 'CHSE' 'HHUM' 'CPSF' 'LBSC' 'AAST' 'BCHM' 'EPIB'
 'ARTT' 'BUAC' 'HLSC' 'CLFS' 'SMLP' 'HLMN' 'MLAW' 'SURV' 'AGNR' 'PLSC'
 'EDPS' 'EDSP' 'ENFP' 'INFM' 'UMEI' 'BEES' 'AMSC' 'EMBA' 'CLAS' 'LATN'
 'IMMR

In [65]:
#Get full names for each department

code_fullname_dict = {}

undergrad_majors = requests.get("https://academiccatalog.umd.edu/undergraduate/approved-courses/")
undergrad_majors_soup = BeautifulSoup(undergrad_majors.text, 'html.parser')

for major in undergrad_majors_soup.find(id="/undergraduate/approved-courses/").find_all("li"):
    majorInfo = major.text.replace('\u200b', '').split(' - ')
    code_fullname_dict[majorInfo[0]] = majorInfo[1].strip()
    
grad_majors = requests.get("https://academiccatalog.umd.edu/graduate/courses/")
grad_majors_soup = BeautifulSoup(grad_majors.text, 'html.parser')

for major in grad_majors_soup.find(id="/graduate/courses/").find_all("li"):
    majorInfo = major.text.replace('\u200b', '').split(' - ')
    if majorInfo[0] not in code_fullname_dict:
        code_fullname_dict[majorInfo[0]] = majorInfo[1].strip()

In [66]:
#Manually add full names for codes that don't have full names yet

for code in uniqueCourses:
    if code not in code_fullname_dict:
        print(code)

code_fullname_dict['BUSO'] = 'Online MBA Program'
code_fullname_dict['BMSO'] = 'Online MS in Business Analytics'
code_fullname_dict['HBUS'] = "Interdisciplinary Business Honors"

BUSO
BMSO
HBUS


In [67]:
#get colleges for some grad school majors

final_code_to_school = {}

gradMajorsSchools = requests.get("https://gradschool.umd.edu/admissions/programs-a-to-z")
gradMajorsSchools_soup = BeautifulSoup(gradMajorsSchools.text, 'html.parser')

for row in gradMajorsSchools_soup.find_all("table")[1].find_all("tr"):
    if len(row.find_all("strong")) == 0:
        final_code_to_school[row.find_all("td")[1].text] = row.find_all("td")[2].text

In [68]:
#Get majors under each school

majors_schools = {}

schools = requests.get("https://academiccatalog.umd.edu/undergraduate/colleges-schools/")
schools_soup =  BeautifulSoup(schools.text, 'html.parser')

links_to_schools = {}

for schools in schools_soup.find_all(id="/undergraduate/colleges-schools/"):
    for majors in schools.find_all("li"):
        if majors.find("a").get("href").count("/") == 4:
            links_to_schools[majors.find("a").text] = majors.find("a").get("href")


for school in links_to_schools:
    majors = requests.get("https://academiccatalog.umd.edu" + links_to_schools[school] + "#degreeprogramstext")
    majors_soup = BeautifulSoup(majors.text, 'html.parser')

    if (len(majors_soup.find_all(id='degreeprogramstextcontainer')) == 0): #Journalism school and office of undergraduate studies
        for section in majors_soup.find_all(id='programstextcontainer'):
            for program in section.find_all("ul")[0]:
                majors_schools[program.text] = school
    else:
        for section in majors_soup.find_all(id='degreeprogramstextcontainer'):
            for program in section.find_all("li"):
                majors_schools[program.text] = school

In [69]:
#Rename long college names to short version

def rename_colleges(dictionary):
    for code in dictionary.keys():
        if 'business' in dictionary[code].lower():
            dictionary[code] = 'BMGT' #'Robert H. Smith School of Business'
        elif 'engineering' in dictionary[code].lower():
            dictionary[code] = 'ENGR' #'A. James Clark School of Engineering'
        elif 'information studies' in dictionary[code].lower():
            dictionary[code] = 'INFO' #'College of Information Studies'
        elif 'computer' in dictionary[code].lower():
            dictionary[code] = 'CMNS' #'College of Computer, Mathematical, and Natural Sciences'
        elif 'arts' in dictionary[code].lower():
            dictionary[code] = 'ARHU'
        elif 'behavioral' in dictionary[code].lower():
            dictionary[code] = 'BSOS'
        elif 'education' in dictionary[code].lower():
            dictionary[code] = 'EDUC'
        elif 'architecture' in dictionary[code].lower():
            dictionary[code] = 'ARCH'
        elif 'health' in dictionary[code].lower():
            dictionary[code] = 'SPHL'
        elif 'agric' in dictionary[code].lower():
            dictionary[code] = 'AGNR'
        elif 'journalism' in dictionary[code].lower():
            dictionary[code] = 'JOUR'
        elif 'inter' in dictionary[code].lower():
            dictionary[code] = 'Office of Undergraduate Studies'

In [70]:
rename_colleges(final_code_to_school)
rename_colleges(majors_schools)

In [71]:
def find_matching_string(string, string_list):
    matches = []
    for element in string_list:
        if string.lower() in element.lower():
            matches.append(element)
    return matches

In [72]:
#biolog - CMNS
#engineer - A James Clark
#ed - College of Education
#college park scholars/gemstone/honors - Office of Undergraduate Studies
#business/MBA - RH Smith
#health - School of Public Health
#agri - College of Agriculture and Natural Resources
#arts - College of Arts and Humanities
#lang/Persian/Russian/Spanish/French/Latin/Portuguese/Korean/Germanic Studies/
#Italian/Hebrew/Greek/america- College of Arts and Humanities

for i in uniqueCourses:
    if i not in final_code_to_school:
        matches = find_matching_string(code_fullname_dict[i], majors_schools.keys())
        if len(matches) > 0:
            final_code_to_school[i] = majors_schools[matches[0]]
        else: #now will need to match keywords of major name to school
            if any(x in code_fullname_dict[i].lower() for x in ['biolog', 'systematics', 'biom', 'machine']):
                final_code_to_school[i] = 'CMNS'
            elif 'engineer' in code_fullname_dict[i].lower():
                final_code_to_school[i] = 'ENGR'
            elif any(x in code_fullname_dict[i].lower() for x in ['ed', 'teach']):
                final_code_to_school[i] = 'EDUC'
            elif any(x in code_fullname_dict[i].lower() for x in ['college park scholars', 'gemstone', 'honors', 'first-year', 'aces', 'civicus', 'design cultures', 'university', 'global', 'maryland']):
                final_code_to_school[i] = 'Office of Undergraduate Studies'
            elif any(x in code_fullname_dict[i].lower() for x in ['business', 'mba', 'accounting', 'entrepreneur', 'manage', 'decision']):
                final_code_to_school[i] = 'BMGT'
            elif any(x in code_fullname_dict[i].lower() for x in ['theatre', 'film','lang','latin', 'germanic', 'immigration', 'art', 'gay']):
                final_code_to_school[i] = 'ARHU'
            elif any(x in code_fullname_dict[i].lower() for x in ['health', 'epid']):
                final_code_to_school[i] = 'SPHL'
            elif any(x in code_fullname_dict[i].lower() for x in ['behavior', 'law']):
                final_code_to_school[i] = 'BSOS'
            elif 'information' in code_fullname_dict[i].lower():
                final_code_to_school[i] = 'INFO'
            elif 'urban' in code_fullname_dict[i].lower(): #architecture
                final_code_to_school[i] = 'ARCH'

In [73]:
for i in uniqueCourses:
    if i not in final_code_to_school:
        print(i)

In [74]:
df_code_dept = pd.DataFrame.from_dict(final_code_to_school, orient='index').reset_index(names='Major').rename(columns={0:'Department'})

In [76]:
df_code_dept.to_csv("../2_clean_data/code_departments.csv", index=False)

In [77]:
df_code_dept

,Major,Department
0,BMAC,BMGT
1,PMAM,ENGR
2,Z115,ENGR
3,ENAE,ENGR
4,PMAE,ENGR
...,...,...
417,CPPL,Office of Undergraduate Studies
418,ENBC,ENGR
419,SLAA,ARHU
420,CPET,Office of Undergraduate Studies
